# Credit Scoring. Con AdaBoosting & GradientBoosting. 
Objetivo: Aplicar un modelo de AdaBoosting y GradientBoosting para predecir, con base en las características del cliente, cuando este completa sus compromisos de pago

### Descripción del conjunto de datos

**status**: Indica si el cliente ha devuleto el pago (1), éxito y en caso contrario (0), fracaso.  (TARGET)

**seniority**: Años de experiencia laboral  (Continua)

**home**: Tipo de vivienda, renta (1), propia (2), y otros.  (Categórica)

**time**: Duración crédito e meses  (Continua)

**age**: Edad del cliente en años  (Continua)

**marital**: Estado civil del cliente. Soltero (1), casado (2), y otros (Categórica)

**records**: Si el cliente tiene registro en el banco (2) y (1) en caso contrario. (Categórica)

**job**: Tipo de empleo. (1) tiempo completo, (2) tiempo parcial y (3) otros. (Categórica)

**expenses**: Gasto mensual del cliente (Continua)

**income**: Ingreso mensual del clientes (Continua)

**assets**: Valor total de los activos del cliente. (Continua)

**debt**: Monto de otras deudas de crédito (Continua)

**amount**: Cantidad de dinero solicitada (Continua)

**price**: Precio del bien que desea adquirir el cliente (Continua)

In [8]:
# Cargamos nuestras librerias
import numpy as np
import pandas as pd

# Entorno SciKit Learn
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier  # Librerias para AdaBoosting y GradientBoosting para Clasificaciòn
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold, RepeatedKFold,cross_val_score, train_test_split
from sklearn.model_selection import GridSearchCV


# Para tratamiento de los datos faltantes
# Reemplazar el dato faltante por la respuesta más frecuente
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer


# Para gráficos
import matplotlib.pyplot as plt
import seaborn as sns

In [9]:
# Lectura de datos
datos=pd.read_csv("creditscoring.csv")

In [10]:
datos.head()

,Status,Seniority,Home,Time,Age,Marital,Records,Job,Expenses,Income,Assets,Debt,Amount,Price
0,1,9,Rentada,60,30,Casado,No,Freelance,73,129.0,0.0,0.0,800,846
1,1,17,Rentada,60,58,Viudo,No,Fijo,48,131.0,0.0,0.0,1000,1658
2,0,10,Propia,36,46,Casado,Si,Freelance,90,200.0,3000.0,0.0,2000,2985
3,1,0,Rentada,60,24,Soltero,No,Fijo,63,182.0,2500.0,0.0,900,1325
4,1,0,Rentada,36,26,Soltero,No,Fijo,46,107.0,0.0,0.0,310,910


In [11]:
# análisis de los datos
datos.describe()

,Status,Seniority,Time,Age,Expenses,Income,Assets,Debt,Amount,Price
count,4454.000000,4454.000000,4454.000000,4454.000000,4454.000000,4420.000000,4407.000000,4436.000000,4454.000000,4454.000000
mean,0.718455,7.986753,46.438707,37.080377,55.573417,130.564253,5403.979351,343.025924,1038.918276,1462.780198
std,0.449803,8.174306,14.655462,10.984598,19.515634,86.376808,11574.418141,1245.991541,474.545999,628.128120
min,0.000000,0.000000,6.000000,18.000000,35.000000,0.000000,0.000000,0.000000,100.000000,105.000000
25%,0.000000,2.000000,36.000000,28.000000,35.000000,80.000000,0.000000,0.000000,700.000000,1117.250000
50%,1.000000,5.000000,48.000000,36.000000,51.000000,120.000000,3000.000000,0.000000,1000.000000,1400.000000
75%,1.000000,12.000000,60.000000,45.000000,72.000000,165.000000,6000.000000,0.000000,1300.000000,1691.500000
max,1.000000,48.000000,72.000000,68.000000,180.000000,959.000000,300000.000000,30000.000000,5000.000000,11140.000000


In [4]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4454 entries, 0 to 4453
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Status     4454 non-null   int64  
 1   Seniority  4454 non-null   int64  
 2   Home       4448 non-null   object 
 3   Time       4454 non-null   int64  
 4   Age        4454 non-null   int64  
 5   Marital    4453 non-null   object 
 6   Records    4454 non-null   object 
 7   Job        4452 non-null   object 
 8   Expenses   4454 non-null   int64  
 9   Income     4420 non-null   float64
 10  Assets     4407 non-null   float64
 11  Debt       4436 non-null   float64
 12  Amount     4454 non-null   int64  
 13  Price      4454 non-null   int64  
dtypes: float64(3), int64(7), object(4)
memory usage: 487.3+ KB


In [12]:
datos.Status.value_counts(normalize=True) # Datos desbalanceados

Status
1    0.718455
0    0.281545
Name: proportion, dtype: float64

# Implementación de un Modelo AdaBoosting

In [13]:
# NOTA: EL modelo de bosques aleatorios no acepta variables objeto
# o tipo cadena para iterar. Entonces, para las variables que son CATEGÓRICAS se crean las dummies.

In [14]:
categoricas=['Home','Marital','Records','Job']
datos=pd.get_dummies(datos,columns=categoricas)  # No se borra la primera categoría. 
datos

,Status,Seniority,Time,Age,Expenses,Income,Assets,Debt,Amount,Price,...,Marital_Divorciado,Marital_Separado,Marital_Soltero,Marital_Viudo,Records_No,Records_Si,Job_Fijo,Job_Freelance,Job_Otro,Job_Tiempo_parcial
0,1,9,60,30,73,129.0,0.0,0.0,800,846,...,False,False,False,False,True,False,False,True,False,False
1,1,17,60,58,48,131.0,0.0,0.0,1000,1658,...,False,False,False,True,True,False,True,False,False,False
2,0,10,36,46,90,200.0,3000.0,0.0,2000,2985,...,False,False,False,False,False,True,False,True,False,False
3,1,0,60,24,63,182.0,2500.0,0.0,900,1325,...,False,False,True,False,True,False,True,False,False,False
4,1,0,36,26,46,107.0,0.0,0.0,310,910,...,False,False,True,False,True,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4449,0,1,60,39,69,92.0,0.0,0.0,900,1020,...,False,False,False,False,True,False,True,False,False,False
4450,1,22,60,46,60,75.0,3000.0,600.0,950,1263,...,False,False,False,False,True,False,True,False,False,False
4451,0,0,24,37,60,90.0,3500.0,0.0,500,963,...,False,False,False,False,True,False,False,False,False,True
4452,1,0,48,23,49,140.0,0.0,0.0,550,550,...,False,False,True,False,True,False,False,True,False,False


In [15]:
# Seleccionamos los atributos y el target
y=datos.Status
del datos["Status"]
X=datos

# Conclusión: el mejor modelo es RandomForestClassifier(max_depth=10,min_samples_split=15)
La métrica de desempeño del mejor modelo es: 0.789 +/- 0.012

# Este modelo final ya nos sirve para hacer predicciones

In [16]:
X_nuevos=pd.DataFrame([[16,36,28,110,130,7000,400,200,1000,0,0,0,0,1,0,1,0,0,0,0,0,1,1,0,0,0]],columns=X.columns)
X_nuevos

,Seniority,Time,Age,Expenses,Income,Assets,Debt,Amount,Price,Home_Familiares,...,Marital_Divorciado,Marital_Separado,Marital_Soltero,Marital_Viudo,Records_No,Records_Si,Job_Fijo,Job_Freelance,Job_Otro,Job_Tiempo_parcial
0,16,36,28,110,130,7000,400,200,1000,0,...,0,0,0,0,0,1,1,0,0,0


In [17]:
# Prediccion
X_nuevos_completo=imput.transform(X_nuevos)  # Utilizamos la técnica para completar los datos faltantes
modelo_final.predict(X_nuevos_completo)      # La utilizamos aunque no tengamos datos faltantes... 

array([1], dtype=int64)